In [191]:
import bs4
import requests
import pandas as pd

In [192]:

def scrape_listings(url):
    rows = []
    r = requests.get(url)

    soup = bs4.BeautifulSoup(r.text, 'html.parser')

    listings = soup.find_all('div', {'class': 'result-item'})
    for element in listings:
        row = {}
        row['url'] = element.find('a').get('href')

        url = 'https://www.exchangeandmart.co.uk' + row['url']
        det_url = requests.get(url)
        soup = bs4.BeautifulSoup(det_url.text, 'html.parser')
        name = soup.find('span', {'class': 'ttl'})
        model = soup.find('span', {'class': 'ttlSup'})
        price = soup.find('span', {'itemprop': 'price'})
        row['name'] = str(name.text).strip()
        row['model'] = str(model.text).strip()
        row['price'] = str(price.text).strip()

        facts = element.find('div', {'class': 'key-details'})
        sayi = 1
        for fact in facts.find_all('p'):
            if(sayi != 4 and sayi != 5):
                row[sayi] = str(fact.text).strip()
            sayi+=1
            #add model and name to rows
        
        det1 = soup.find('div', {'class': 'adDetails'})
        det2 = soup.find('div', {'class': 'adDetsColumn'})

        for cell in det1.find_all('div', {'class': 'adDetsItem'}):
            row[sayi] = str(cell.text).strip()
            sayi+=1
        counter = 1
        
        for span in det2.find_all('span'):
            if counter%2 == 0:
                row[sayi] = str(span.text).strip()
                sayi+=1
            counter        

        rows.append(row)

    return pd.DataFrame(rows)


In [193]:
# Solution
df = scrape_listings('https://www.exchangeandmart.co.uk/used-cars-for-sale/toyota/any-distance-from-wc1e-6dp/fuel-hybrid/page')
df

,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30919027,Toyota RAV4,4 2.5 VVT-i Hybrid Excel 5dr CVT SUV,"28,179",2019,Automatic,49.6 mpg,Hybrid,2019,2.5L,"11,923 miles",Hybrid,Automatic,Grey,SUV,49.6 mpg
1,/ad/30854130,Toyota Yaris,s 1.5 Hybrid Icon Tech 5dr CVT Hatchback,"15,250",2019,Automatic,76.3 mpg,Hybrid,2019,1.5L,"12,868 miles",Hybrid,Automatic,Other,Hatchback,76.3 mpg
2,/ad/30822867,Toyota Corolla,TOURING SPORT 2.0 VVT-i Hybrid Excel 5dr CVT E...,"27,199",2022,Other,52.3 mpg,Hybrid,2022,1987cc,"8,329 miles",Hybrid,N/A,Blue,Estate,52.3 mpg
3,/ad/30885817,Toyota Corolla,1.8 VVT-i Hybrid Design 5dr CVT Hatchback,"21,049",2022,Other,57.7 mpg,Hybrid,2022,1798cc,"8,832 miles",Hybrid,N/A,White,Hatchback,57.7 mpg
4,/ad/30815260,Toyota Corolla,1.8 VVT-i Hybrid GR Sport 5dr CVT Hatchback,"22,199",2022,Other,57.7 mpg,Hybrid,2022,1798cc,"9,518 miles",Hybrid,N/A,Black,Hatchback,57.7 mpg
5,/ad/30919818,Toyota Corolla,2.0 VVT-i Hybrid GR Sport 5dr CVT Hatchback,"22,449",2021,Other,53.3 mpg,Hybrid,2021,1987cc,"10,029 miles",Hybrid,N/A,Silver,Hatchback,53.3 mpg
6,/ad/30877493,Toyota Corolla,1.8 VVT-i Hybrid Icon 5dr CVT Hatchback,"18,299",2022,Other,62.8 mpg,Hybrid,2022,1798cc,"10,900 miles",Hybrid,N/A,White,Hatchback,62.8 mpg
7,/ad/30869987,Toyota C-HR,1.8 Hybrid Icon 5dr CVT [Tech] SUV,"18,799",2020,Other,58.9 mpg,Hybrid,2020,1798cc,"14,800 miles",Hybrid,N/A,Silver,SUV,58.9 mpg
8,/ad/30927849,Toyota Corolla,1.8 VVT-i Hybrid GR Sport 5dr CVT Hatchback,"20,349",2021,Other,57.7 mpg,Hybrid,2021,1798cc,"17,018 miles",Hybrid,N/A,Red,Hatchback,57.7 mpg
9,/ad/30822861,Toyota C-HR,1.8 Hybrid Dynamic 5dr CVT SUV,"20,999",2020,Other,58.9 mpg,Hybrid,2020,1798cc,"18,477 miles",Hybrid,N/A,White,SUV,58.9 mpg


In [194]:
def scrape_all(pages):
    url = 'https://www.exchangeandmart.co.uk/used-cars-for-sale/toyota/fuel-hybrid/page'
    df = pd.DataFrame()
    for page in pages:
        df = pd.concat([df, scrape_listings(url + str(page))], ignore_index=True)
    return df

In [196]:
df_large = scrape_all(range(1, 5))
df_large

,url,name,model,price,1,2,3,6,7,8,9,10,11,12,13,14
0,/ad/30701733,Toyota Corolla,1.8 Hybrid Design 5dr CVT Hatchback,"28,550",2023,Automatic,61.4 mpg,Hybrid,2023,1798cc,3 miles,Petrol Hybrid,Automatic,White,Hatchback,61.4 mpg
1,/ad/30778327,Toyota C-HR,2.0 Hybrid GR Sport 5dr CVT SUV,"36,500",2023,Automatic,53.3 mpg,Hybrid,2023,1987cc,3 miles,Petrol Hybrid,Automatic,Unknown,SUV,53.3 mpg
2,/ad/30922212,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Blue,Hatchback,68.9 mpg
3,/ad/30922210,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Blue,Hatchback,68.9 mpg
4,/ad/30929826,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"23,473",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,4 miles,Petrol Hybrid,Automatic,Red,Hatchback,68.9 mpg
5,/ad/30904720,Toyota C-HR,2.0 Hybrid GR Sport 5dr CVT (Safety Pack) SUV,"42,995",2023,Automatic,57.7 mpg,Hybrid,2023,1987cc,5 miles,Petrol Hybrid,Automatic,Unknown,SUV,57.7 mpg
6,/ad/30930099,Toyota C-HR,2.0 Hybrid GR Sport 5dr CVT (Safety Pack) SUV,"44,995",2023,Automatic,57.7 mpg,Hybrid,2023,1987cc,5 miles,Petrol Hybrid,Automatic,Unknown,SUV,57.7 mpg
7,/ad/30778330,Toyota RAV4,4 2.5 PHEV Dynamic 5dr CVT SUV,"48,995",2023,Automatic,Tax: £170,2023,2487cc,5 miles,Petrol Plugin Hybrid,Automatic,Unknown,SUV,NaN,NaN
8,/ad/30516258,Toyota Yaris,1.5 Hybrid Design 5dr CVT (Nav) Hatchback,"22,973",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,6 miles,Petrol Hybrid,Automatic,Unknown,Hatchback,68.9 mpg
9,/ad/30771752,Toyota Yaris,1.5 Hybrid Icon 5dr CVT Hatchback,"22,995",2023,Automatic,68.9 mpg,Hybrid,2023,1490cc,6 miles,Petrol Hybrid,Automatic,White,Hatchback,68.9 mpg
